# Validate `Humor-Research/humor-detection-comb-23` on a TSV file

This notebook:

1. Loads a TSV file.
2. Feeds the **`prediction`** column (text) into the model `Humor-Research/humor-detection-comb-23`.
3. Adds per-row probabilities and predicted labels.
4. Summarizes:
   - **average certainty for predicted non-jokes (label 0)**
   - **average certainty for predicted jokes (label 1)**
   - **count of predicted non-jokes and jokes**

> Note: The Humor-Research model repositories typically do not include tokenizer files, so we load the tokenizer from `roberta-base` (this is also the usage shown by the authors).


In [ ]:
# If needed, uncomment and run once:
# %pip install -q torch transformers pandas tqdm

In [1]:
import os
import pandas as pd
import torch
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification, RobertaTokenizerFast

print("torch:", torch.__version__)
import transformers
print("transformers:", transformers.__version__)

/Users/antonii/nlp_repo/MWAHAHA_Competition/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch: 2.9.1
transformers: 4.57.3


In [41]:
# === Configuration ===

TSV_FILE = "task-a-title_predictions_base 6 (all jokes generated)"
TSV_PATH = f"../../experiment_results_good/{TSV_FILE}.tsv"   # <-- set your TSV path here
TEXT_COLUMN = "prediction"             # <-- the column to score with the model

BATCH_SIZE = 32
MAX_LENGTH = 256  # 512 is fine too; 256 is usually enough for short jokes

OUT_DIR = "results_exp_good"
os.makedirs(OUT_DIR, exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

MODEL_ID = "Humor-Research/humor-detection-comb-23"
TOKENIZER_ID = "roberta-base"  # tokenizer comes from roberta-base

Device: cpu


## Load TSV

In [42]:
if not os.path.exists(TSV_PATH):
    raise FileNotFoundError(f"TSV file not found: {TSV_PATH}")

df = pd.read_csv(TSV_PATH, sep="\t")
print("Rows:", len(df))
print("Columns:", list(df.columns))

if TEXT_COLUMN not in df.columns:
    raise KeyError(f"Column '{TEXT_COLUMN}' not found. Available columns: {list(df.columns)}")

df.head()

Rows: 1100
Columns: ['Unnamed: 0', 'headline', 'noun1', 'noun2', 'prediction']


,Unnamed: 0,headline,noun1,noun2,prediction
0,0,Ryanair to cut 1 million more passenger seats ...,seat,spain,Ryanair's cutting seats in Spain? Pricey irony...
1,1,"Looted by Nazis, a 17th-Century Painting Resur...",painting,long,A 17th-century painting so long it needs glass...
2,2,Analysis: Spotlight on childcare reforms revea...,spotlight,reform,"Spotlight often shines where there's no need, ..."
3,3,Do body wipes actually work? Experts weigh in,wipe,body,Experts weigh in: Do body wipes actually work?...
4,4,Is Meghan’s Netflix series another ‘exercise i...,exercise,series,Is 'The Meghan Show' really just another exerc...


## Load model + tokenizer

In [43]:
tokenizer = RobertaTokenizerFast.from_pretrained(TOKENIZER_ID)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID)
model.to(DEVICE)
model.eval()

print("Model loaded:", MODEL_ID)

Model loaded: Humor-Research/humor-detection-comb-23


## Score the TSV (probabilities and predictions)

We compute:

- `p_not_joke` = probability of label 0  
- `p_joke` = probability of label 1  
- `model_pred` = argmax label (0 or 1)  
- `certainty` = probability of the predicted label


In [44]:
@torch.inference_mode()
def score_texts(texts):
    all_p0 = []
    all_p1 = []
    for start in tqdm(range(0, len(texts), BATCH_SIZE), desc="Scoring"):
        batch_texts = texts[start:start+BATCH_SIZE]
        enc = tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=MAX_LENGTH,
        ).to(DEVICE)

        logits = model(**enc).logits  # [batch, 2]
        probs = torch.softmax(logits, dim=-1).detach().cpu()  # [batch, 2]
        all_p0.extend(probs[:, 0].tolist())
        all_p1.extend(probs[:, 1].tolist())
    return all_p0, all_p1

texts = df[TEXT_COLUMN].astype(str).tolist()
p0, p1 = score_texts(texts)

df["p_not_joke"] = p0
df["p_joke"] = p1
df["model_pred"] = (df["p_joke"] > df["p_not_joke"]).astype(int)
df["certainty"] = df.apply(lambda r: r["p_joke"] if r["model_pred"] == 1 else r["p_not_joke"], axis=1)

df.head()

Scoring: 100%|██████████| 35/35 [00:07<00:00,  4.72it/s]


,Unnamed: 0,headline,noun1,noun2,prediction,p_not_joke,p_joke,model_pred,certainty
0,0,Ryanair to cut 1 million more passenger seats ...,seat,spain,Ryanair's cutting seats in Spain? Pricey irony...,0.097718,0.902282,1,0.902282
1,1,"Looted by Nazis, a 17th-Century Painting Resur...",painting,long,A 17th-century painting so long it needs glass...,0.013126,0.986874,1,0.986874
2,2,Analysis: Spotlight on childcare reforms revea...,spotlight,reform,"Spotlight often shines where there's no need, ...",0.990283,0.009717,0,0.990283
3,3,Do body wipes actually work? Experts weigh in,wipe,body,Experts weigh in: Do body wipes actually work?...,0.210402,0.789598,1,0.789598
4,4,Is Meghan’s Netflix series another ‘exercise i...,exercise,series,Is 'The Meghan Show' really just another exerc...,0.762909,0.237091,0,0.762909


## Summary: average certainties and counts

In [45]:
# Counts
count_not_joke = int((df["model_pred"] == 0).sum())
count_joke = int((df["model_pred"] == 1).sum())

# Average certainty for each predicted class:
avg_not_joke_certainty = float(df["p_not_joke"].mean()) if count_not_joke else float("nan")
avg_joke_certainty = float(df["p_joke"].mean()) if count_joke else float("nan")

summary = pd.DataFrame([
    {"predicted_label": 0, "label_name": "not_joke", "count": count_not_joke, "avg_certainty": avg_not_joke_certainty},
    {"predicted_label": 1, "label_name": "joke", "count": count_joke, "avg_certainty": avg_joke_certainty},
])

summary

,predicted_label,label_name,count,avg_certainty
0,0,not_joke,418,0.409027
1,1,joke,682,0.590973


In [46]:
print("Predicted not_joke count:", count_not_joke, "| average certainty:", avg_not_joke_certainty)
print("Predicted joke count    :", count_joke,     "| average certainty:", avg_joke_certainty)

Predicted not_joke count: 418 | average certainty: 0.4090266063841144
Predicted joke count    : 682 | average certainty: 0.5909733923721466


## Save outputs

- `output/comb23_scored_with_summary.tsv` – your original TSV plus probabilities and predictions + summary table (counts and average certainties)

In [47]:
# Save outputs (single file)

# We will write everything to ONE TSV file:
# 1) the scored rows (original TSV + probabilities + predictions)
# 2) a blank line
# 3) a small summary block with counts + average certainties

single_path = os.path.join(OUT_DIR, f"{TSV_FILE}_scored_with_summary.tsv")

# 1) write scored rows
df.to_csv(single_path, sep="\t", index=False)

# 2) append summary block
with open(single_path, "a", encoding="utf-8") as f:
    f.write("\n\n")
    f.write("# summary\n")
    f.write(f"# predicted_not_joke_count\t{count_not_joke}\n")
    f.write(f"# predicted_joke_count\t{count_joke}\n")
    f.write(f"# avg_not_joke_certainty\t{avg_not_joke_certainty}\n")
    f.write(f"# avg_joke_certainty\t{avg_joke_certainty}\n")
    f.write("\n")
    # also append the summary table as TSV
    summary.to_csv(f, sep="\t", index=False)

print("Saved single TSV (scored + summary):", single_path)


Saved single TSV (scored + summary): results_exp_good/task-a-title_predictions_base 6 (all jokes generated)_scored_with_summary.tsv
